In [2]:
import requests
import json 
import sys


In [ ]:
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio import AlignIO

## Aceso a todas las variantes de un gen a través de la API de Ensembl

In [135]:
def acceso_variantes_ensembl (identificador_gen, especie):
    """ 
    DESCRIPCIÓN DE LA FUNCIÓN"""
    
    rest_api = "https://rest.ensembl.org"
    extension_variantes = f"/overlap/id/{identificador_gen}?feature=variation;species={especie}"

    solicitud = requests.get(rest_api+extension_variantes, headers={"Content-Type" : "application/json"})


    if not solicitud.ok:
        solicitud.raise_for_status()
        sys.exit()

    variantes = solicitud.json()
    
    return variantes



## Obtención de las variantes patogénicas a partir de las variantes que hemos obtenido de la API de Ensembl

In [136]:
def obtener_variantes_patogenicas (identificador_gen, especie):
    """
    DESCRIPCIÓN DE LA FUNCIÓN
    """

    variantes = acceso_variantes_ensembl (identificador_gen, especie)
    variantes_patogenicas = []

    for var in variantes:
        if "clinical_significance" in var and "pathogenic" in var["clinical_significance"]:
            variantes_patogenicas.append(var)


    return variantes_patogenicas
            

## Obtención de las variantes benignas a partir de las variantes que hemos obtenido de la API de Ensembl

In [137]:
def obtener_variantes_benignas (identificador_gen, especie):
    """
    DESCRIPCIÓN DE LA FUNCIÓN
    """

    variantes = acceso_variantes_ensembl (identificador_gen, especie)
    variantes_benignas = []

    for var in variantes:
        if "clinical_significance" in var and "benign" in var["clinical_significance"]:
            variantes_benignas.append(var)


    return variantes_benignas
            

## Obtención de la localización de las variantes

In [159]:
def obtener_localizacion_variantes (variantes):

    diccionario_posiciones_variantes = {}

    for var in variantes:
        inicio = var["start"]
        final = var["end"]
        longitud = final - (inicio - 1)
        diccionario_posiciones_variantes[var["id"]] = [inicio, final, longitud]
    
    return diccionario_posiciones_variantes
        

## Obtención de los exones

In [148]:
def obtener_exones_gen (identificador_transcrito, especie):
    
    rest_api = "https://rest.ensembl.org"
    extension_exones = f"/lookup/id/{identificador_transcrito}?expand=1&species={especie}"

    solicitud = requests.get(rest_api+extension_exones, headers={"Content-Type" : "application/json"})

    if not solicitud.ok:
        solicitud.raise_for_status()
        sys.exit()

    exones = solicitud.json()

    return exones["Exon"]

    

## Obtención de la secuencia de los exones en formato fasta

In [413]:
def obtener_secuencia_exones_fasta (identificador_transcrito, especie, nombre_archivo_fasta):
    exones = obtener_exones_gen (identificador_transcrito, especie)

    lista_secuencias = []

    contador = 1

    for ex in exones:
        identificador_exon = ex["id"]

        rest_api = "https://rest.ensembl.org"
        extension_secuencia_exon = f"/sequence/id/{identificador_exon}?content-type=text/x-fasta"

        solicitud2 = requests.get(rest_api+extension_secuencia_exon, headers={ "Content-Type" : "application/json"})

        if not solicitud2.ok:
            solicitud2.raise_for_status()
            sys.exit()

        exon = solicitud2.json()
        
        if ex["strand"] == -1:
            secuencia_fasta = SeqRecord (Seq(exon["seq"]).complement())
            
        else:
            secuencia_fasta = SeqRecord (Seq(exon["seq"]))
            
        secuencia_fasta.id = exon["id"]
        secuencia_fasta.name = exon["id"] + "exon" + str(contador)
        secuencia_fasta.description = exon["desc"]  + " exon " + str(contador)
        lista_secuencias.append(secuencia_fasta)

        contador +=1

    SeqIO.write(lista_secuencias, nombre_archivo_fasta, "fasta")

    print ("Se han guardado las secuencias correctamente")
    
    return exones


## Obtención de la localización de los exones

In [28]:
def obtener_localizacion_exones (exones):
    
    diccionario_posiciones_exones = {}

    for exon in exones:
        inicio = exon["start"]
        final = exon["end"]
        longitud = final - inicio
        diccionario_posiciones_exones[exon["id"]] = [inicio, final, longitud]
    
    return diccionario_posiciones_exones
        
    

## Obtención de la CDS (coding sequence) de un gen

In [37]:
def obtener_cds_gen (identificador_transcrito, especie):
    rest_api = "https://rest.ensembl.org"
    extension_cds = f"/sequence/id/{identificador_transcrito}?type=cds;species={especie}"

    solicitud = requests.get(rest_api+extension_cds, headers={"Content-Type" : "application/json"})

    if not solicitud.ok:
        solicitud.raise_for_status()
        sys.exit()

    secuencia_cds = solicitud.json()

    return secuencia_cds

In [ ]:
secuencia_fasta = SeqRecord (Seq(exon["seq"]))
            
        secuencia_fasta.id = exon["id"]
        secuencia_fasta.name = exon["id"] + "exon" + str(contador)
        secuencia_fasta.description = exon["desc"]  + " exon " + str(contador)
        lista_secuencias.append(secuencia_fasta)

        contador +=1

    SeqIO.write(lista_secuencias, nombre_archivo_fasta, "fasta")

    print ("Se han guardado las secuencias correctamente")
    
    return exones


In [55]:
def obtener_cds_gen_fasta (identificador_transcrito, especie, nombre_archivo_fasta):
    region_codificante = obtener_cds_gen (identificador_transcrito, especie)
    lista_fasta = []
    
    cds_fasta = SeqRecord (Seq(region_codificante["seq"]), id = region_codificante["id"])
    cds_fasta.id = region_codificante["id"]
    cds_fasta.name = region_codificante["id"] + " región codificante"
    cds_fasta.description = region_codificante["desc"]
    print (cds_fasta)

    lista_fasta.append(cds_fasta)

    SeqIO.write(lista_fasta, nombre_archivo_fasta, "fasta")

    print ("La región CDS se ha guardado correctamente")

    return region_codificante
  


In [56]:
obtener_cds_gen_fasta ("ENST00000269305", "human", "data_TP53/region_CDS.fasta")

ID: ENST00000269305
Name: ENST00000269305 región codificante
Number of features: 0
Seq('ATGGAGGAGCCGCAGTCAGATCCTAGCGTCGAGCCCCCTCTGAGTCAGGAAACA...TGA')


AttributeError: 'NoneType' object has no attribute 'replace'

## Obtener un diccionario con todas las variantes de un determinado exón 

In [443]:
def obtener_variantes_exon (variantes, identificador_exon, exones):
    for ex in exones:
        if ex["id"] == identificador_exon:
            inicio_exon = ex["start"]
            final_exon = ex["end"]

            diccionario_variantes_exon = {}

            for var in variantes:
                inicio_variante = var["start"]

                if inicio_exon <= inicio_variante <= final_exon:
                    diccionario_variantes_exon[var["id"]] = var

    return diccionario_variantes_exon
    

## Obtención de la secuencia de las variantes, tanto patogénicas, como benignas

Como al utilizar la API de Ensembl para obtener las variantes no obtenemos la secuencia completa del gen con estas variaciones, sino que solo obtenemos los alelos que han sufrido la variación lo que vamos a hacer es lo siguiente:

1. A partir del diccionario de las variantes que corresponden a un determinado exón, vamos a identificar el tipo de variación que ha sufrido la secuencia, una inserción (el alelo de referencia es más corto que el alternativo), una delección (el alelo de referencia tiene una mayor longitud que el alternativo) y un reemplazo (el alelo de referencia y el alternativo presentan la misma longitud). 
   
3. Realizamos la modificación de la secuencia del exón con el alelo mutado.  

In [29]:
def obtener_secuencia_variantes_exon (diccionario_variantes_exon, exones, exon_fasta):

    for ex in exones:
        if ex["id"] == exon_fasta.id:
            inicio_exon = ex["start"]
            final_exon = ex["end"]
            sec_exon_fasta = exon_fasta.seq
            
            secs =[]
            
            for var in diccionario_variantes_exon.values():
                if len(var["alleles"][0]) > len(var["alleles"][1]):
                    longitud_variante = len(var["alleles"][0])
                    inicio_variante = final_exon - var["start"] - len(var["alleles"][0])
                    secuencia_variante = sec_exon_fasta[:inicio_variante+1]+var["alleles"][1]+sec_exon_fasta[inicio_variante+longitud_variante+1:]
                    secs.append ([var["id"],secuencia_variante])

                elif len(var["alleles"][0]) < len(var["alleles"][1]):
                    longitud_variante = len(var["alleles"][0])
                    inicio_variante = final_exon - var["start"] - len(var["alleles"][0])
                    secuencia_variante = sec_exon_fasta[:inicio_variante+1]+var["alleles"][1]+sec_exon_fasta[inicio_variante+longitud_variante+1:]
                    secs.append ([var["id"],secuencia_variante])

                else:
                    longitud_variante = len(var["alleles"][0])
                    inicio_variante = final_exon - var["start"]
                    secuencia_variante = sec_exon_fasta[:inicio_variante]+var["alleles"][1]+sec_exon_fasta[inicio_variante+longitud_variante:]
                    secs.append ([var["id"],secuencia_variante])
                    
                    

    return secs
        

## Obtención de la secuencia de las variantes patogénicas en formato fasta

Como las variantes que hemos obtenido en un primer paso de obtención de variantes sabemos que corresponden a humanos, no necesitamso especificar la especie en este caso.

Vamos a crear un objeto tipo SeqRecord con los datos de la variante y lo vamos a almacenar en un archivo de tipo fasta

In [446]:
def obtener_secuencia_variantes_fasta (diccionario_variantes_exon, secuencia_variantes, nombre_archivo_fasta, exon_fasta):
    lista_secuencias = []

    for var in diccionario_variantes_exon.values():
        for seq in secuencia_variantes:
            if var["id"] == seq[0]:
                secuencia_fasta = SeqRecord (Seq(seq[1]))
                secuencia_fasta.id = var["id"]
                secuencia_fasta.name = var["id"]
                secuencia_fasta.description = str(var["clinical_significance"]) + str (exon_fasta.id) + str (exon_fasta.description)
                lista_secuencias.append (secuencia_fasta)

    SeqIO.write(lista_secuencias, nombre_archivo_fasta, "fasta")

    print ("Las secuencias de las variantes se han guardado en formato fasta")
                
        
    
    